In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from CRUD_Animals import AnimalShelter
import base64

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

projection = {'_id': False}
df = pd.DataFrame.from_records(shelter.read({}, projection)) 
image_filename = 'Grazioso-Salvare-Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################


In [3]:
app = JupyterDash('SimpleExample')

app.layout = html.Div([

    html.Center(html.B(html.H1('GRAZAIOSO SALVARE'))),
    html.Center("Jonathan Haro"),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),

    html.Div(className='row',
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
                    html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                    html.Button(id='submit-button-four', n_clicks=0, children='Reset'),
                ]
    ),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        editable=False,
        row_selectable=True,
        row_deletable=False,
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 5,

    ),

    html.Br(), # wrapper for <br> line break
    html.Hr(), # wrapper for <hr> line new paragraph

    html.Div(className='row',
         style={'display' : 'flex'},
            children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',

                    ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
            ]
    )  
])

############################################
#Interaction Between Components / Controller
############################################
def disasterRescueQuery():
    return {
        "animal_type":"Dog",
        "breed":{"$in":["Doberman Pinscher","German Shepard","Golden Retriever","Bloodhound","Rottweiler"]
        }, 
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},
        "age_upon_outcome_in_weeks":{"$lte":300}
    }

def waterRescueQuery():
    return {
        "animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]
        }, 
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},
        "age_upon_outcome_in_weeks":{"$lte":156}
    }

def mountainRescueQuery():
    return {
        "animal_type":"Dog",
        "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]
        }, 
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},
        "age_upon_outcome_in_weeks":{"$lte":156}
    }

@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')]
)

def on_click(bt1, bt2, bt3, bt4):

    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
        df = pd.DataFrame.from_records(shelter.read({}, projection))
    #button 1    
    elif (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4)):
        df = pd.DataFrame.from_records(shelter.read(waterRescueQuery(), projection))
    #button 2
    elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
        df = pd.DataFrame.from_records(shelter.read(mountainRescueQuery(), projection))
    #button 3
    elif (int(bt3) > int(bt1) and int(bt3) > int(bt2) and int(bt3) > int(bt4)):
        df = pd.DataFrame.from_records(shelter.read(disasterRescueQuery(), projection))
    #button 4
    elif (int(bt4) > int(bt1) and int(bt4) > int(bt2) and int(bt4) > int(bt3)):
        df = pd.DataFrame.from_records(shelter.read({}, projection))

    return df.to_dict('records')


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dl.Map(style={'width': '800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

# @app.callback(
#     Output('graph-id', "children"), 
#     [Input('datatable-id', "data")])

# def update_graphs(viewData):
#     dff = pd.DataFrame.from_dict(viewData)
#     df1 = dff.groupby(["breed"]).count().reset_index()

#     return [
#         dcc.graph(
#             figure = px.bar(df1, y=dff.groupby(["breed"]).size(), x = "breed")
#         )
#     ]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, values=values, names=names, title='Percentage of breeds available')
        )    
    ]

app